In [1]:
import EDA_Poverty as data
import pandas as pd 
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Flatten,Concatenate,Input,Embedding
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt


2024-10-29 18:17:01.395813: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-29 18:17:01.405625: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730206021.417703   22804 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730206021.420919   22804 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-29 18:17:01.433037: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)
import warnings
warnings.filterwarnings('ignore')

In [3]:
Poverty_1=data.get_poverty_1_data()
Poverty_1

<class 'pandas.core.frame.DataFrame'>
Index: 4411 entries, 0 to 4876
Columns: 108 entries, country to p50_p10_index
dtypes: float64(102), int64(3), object(3)
memory usage: 3.7+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 4411 entries, 0 to 4876
Columns: 108 entries, country to p50_p10_index
dtypes: float64(102), int64(3), object(3)
memory usage: 3.7+ MB


,country,year,hc_ratio,international_poverty_gap,$1_poverty_gap
0,Albania,1996,0.920669,0.140051,0.011726
1,El Salvador,2009,7.419960,1.988661,0.152806
2,El Salvador,2010,6.345635,1.617116,0.142835
3,El Salvador,2011,5.340929,1.250314,0.070231
4,El Salvador,2012,4.637148,1.117315,0.067280
...,...,...,...,...,...
3638,Argentina,2007,1.723668,0.663743,0.293898
3639,Argentina,2006,2.118256,0.796947,0.259502
3640,Argentina,2005,2.631179,0.895969,0.315964
3641,Argentina,2003,5.279263,1.944457,0.679037


In [4]:
country_encoder=LabelEncoder()
country_encoded=country_encoder.fit_transform(Poverty_1[['country']])
country_encoded_df=pd.DataFrame(country_encoded).rename({0:'country'},axis=1)
X=pd.concat([country_encoded_df['country'],Poverty_1[['year','hc_ratio','international_poverty_gap','$1_poverty_gap']]],axis=1)

In [5]:
X

,country,year,hc_ratio,international_poverty_gap,$1_poverty_gap
0,0,1996,0.920669,0.140051,0.011726
1,42,2009,7.419960,1.988661,0.152806
2,42,2010,6.345635,1.617116,0.142835
3,42,2011,5.340929,1.250314,0.070231
4,42,2012,4.637148,1.117315,0.067280
...,...,...,...,...,...
3638,3,2007,1.723668,0.663743,0.293898
3639,3,2006,2.118256,0.796947,0.259502
3640,3,2005,2.631179,0.895969,0.315964
3641,3,2003,5.279263,1.944457,0.679037


In [6]:
y=Poverty_1[['international_poverty_gap','$1_poverty_gap']]
y

,international_poverty_gap,$1_poverty_gap
0,0.140051,0.011726
1,1.988661,0.152806
2,1.617116,0.142835
3,1.250314,0.070231
4,1.117315,0.067280
...,...,...
3638,0.663743,0.293898
3639,0.796947,0.259502
3640,0.895969,0.315964
3641,1.944457,0.679037


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
num_countries = len(X['country'].unique())
year_shape = (X[['year']].shape[1],)
hc_ratio_shape = (X[['hc_ratio']].shape[1],)

train_country = X_train['country']  # First column: country
train_year = X_train['year']     # Second column: year
train_hc_ratio = X_train['hc_ratio'] # Third column: headcount

test_country = X_test['country']  # First column: country
test_year = X_test['year']     # Second column: year
test_hc_ratio = X_test['hc_ratio'] # Third column: headcount


In [9]:
def create_model(num_countries,year_shape,hc_ratio_shape):
    country_input = Input(shape=(1,), name='country')
    year_input = Input(shape=(1,), name='year')
    hc_ratio_input=Input(shape=(1,),name='hc_ratio')
    
    country_embedding = Embedding(input_dim=num_countries, output_dim=15)(country_input)
    country_embedding = Flatten()(country_embedding)
    
    
    concatenated_inputs = Concatenate()([country_embedding, year_input, hc_ratio_input])
    
    dense_layer = Dense(128, activation='relu')(concatenated_inputs)
    dense_layer = Dense(64, activation='relu')(dense_layer)
    dense_layer = Dense(32, activation='relu')(dense_layer)
    dense_layer = Dense(16, activation='relu')(dense_layer)
    output = Dense(2, activation='linear')(dense_layer)
    
    model=Model(inputs=[country_input,year_input,hc_ratio_input],outputs=output)
    
    model.compile(optimizer='adam', loss='mean_squared_error',metrics=['mae'])
    
    return model


In [10]:
model=create_model(num_countries,year_shape,hc_ratio_shape)

model.summary()

W0000 00:00:1730206023.149054   22804 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ country             │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 1, 15)     │      2,445 │ country[0][0]     │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 15)        │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ year (InputLayer)   │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hc_ratio            │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 17)        │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ year[0][0],       │
│                     │                   │            │ hc_ratio[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │      2,304 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      8,256 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 32)        │      2,080 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 16)        │        528 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 2)         │         34 │ dense_3[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 15,647 (61.12 KB)

 Trainable params: 15,647 (61.12 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.fit([train_country, train_year, train_hc_ratio], y_train, epochs=1000, batch_size=32, validation_split=0.2,callbacks=EarlyStopping(
                                                                                monitor='val_loss',  
                                                                                 patience=10,
                                                                                restore_best_weights=True  
                                                                                    )
          )

Epoch 1/1000
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 6437.0649 - mae: 44.0362 - val_loss: 50.8834 - val_mae: 3.5655
Epoch 2/1000
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 46.1577 - mae: 3.6596 - val_loss: 34.6805 - val_mae: 3.7965
Epoch 3/1000
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 35.9146 - mae: 3.3510 - val_loss: 25.7255 - val_mae: 3.2610
Epoch 4/1000
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 21.5095 - mae: 2.5730 - val_loss: 16.7569 - val_mae: 1.9081
Epoch 5/1000
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14.9466 - mae: 2.1092 - val_loss: 11.8705 - val_mae: 1.7582
Epoch 6/1000
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 11.3528 - mae: 1.7644 - val_loss: 11.4163 - val_mae: 2.4106
Epoch 7/1000
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9.0660 - mae: 1.5130 - val_loss: 7.7947 - val_mae: 1.3203
Epoch 8/1000
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 955us/step - loss: 5.3489 - mae: 1.1401 - val_loss: 6.7809 - val_mae: 1.0992
Epoch 9/1000
73/73 ━━━━━━━━━━━━━━━━━━━━

In [12]:

predictions = model.predict([test_country, test_year, test_hc_ratio])
predictions

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[ 2.8089581e+01,  1.2278432e+01],
       [ 2.9608247e-01,  4.3256778e-01],
       [ 8.9322710e+00,  1.6477441e+00],
       ...,
       [ 1.4379113e+01,  2.3542340e+00],
       [-2.1842020e-02,  2.1305364e-01],
       [ 4.5222750e+00,  8.2026309e-01]], dtype=float32)

In [13]:
from sklearn.metrics import mean_squared_error, r2_score

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)

print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")

RMSE: 1.2540180606664286
R-squared: 0.8384839296340942


# Preparing model for deployment

In [14]:
from tensorflow.keras.models import load_model
import pickle

In [15]:
filename='poverty_predictor.h5'
model.save(filename)

In [17]:
model = load_model('poverty_predictor.h5')

with open('/home/savyasanchisharma/Poverty-Prediction-Model/Web App/Trained Models and Encoder/country_encoder.pkl', 'rb') as file:
    country_encoder = pickle.load(file)

def predict_poverty(country, year, hc_ratio):
    country_code=country_encoder.transform([country])
    country_code = np.array([country_code]).reshape(-1, 1)
    year = np.array([year]).reshape(-1, 1)
    hc_ratio = np.array([hc_ratio]).reshape(-1, 1)
    
    # Make predictions using the model
    prediction = model.predict([country_code, year, hc_ratio])
    
    return prediction[0][0],prediction[0][1]  # Return the predicted value

In [18]:
predict_poverty('Albania',2020,90)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step


(45.93731, 18.801865)